## Granger's Causality Test

We have a lot of predictors. To prune them, we'll (maybe? still haven't decided) consider the results of Granger's Causality test. This test essentially evaluates whether adding a variable to an autoregression of another variable improves the model's predictive power (measured via the p-values on the coefficients of the lags of the added variable).

Load the (differenced, stationary) data. 

In [ ]:
# PACKAGES
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests


data = pd.read_csv("../data/data_differenced_wpk.csv")
data.set_index('date', inplace=True)


In [ ]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grange = grangers_causation_matrix(data, variables = data.columns)

c:\Users\wpken\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\wpken\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\wpken\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\wpken\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\wpken\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  

KeyError: 1

In [9]:
grange.iloc[0, :]

gdp_yoy_x               1.0000
orders_x                0.0009
employment_x            0.0000
consumer_sentiment_x    0.1051
unemploy_claims_x       0.0000
bus_outlook_x           0.0005
auto_sales_x            0.0000
construction_x          0.1220
itrade_x                0.0000
wtrade_x                0.0001
Name: gdp_yoy_y, dtype: float64

All of these values are p-values, evaluating whether x "causes" y (this test doesn't really evaluate causation, it's more about predictive power). 

Using a 5% level cutoff for our p-values, we fail to reject only for consumer_sentiment and construction. This actually kind of makes sense -- consumer sentiment can be fickle and doesn't always reflect consumer's actual actions (survey bias) and construction is relatively small part of total US economic activity. 